In [1]:
from conftest import setup_project
setup_project()

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import chex
import numpy as np
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import tqdm
from JAxtar.hash import hash_func_builder, HashTable
from JAxtar.bgpq import BGPQ, HashTableIdx_HeapValue
from JAxtar.astar import astar_builder
from functools import partial
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic

/tmp/ipykernel_1256553/2469263457.py:8: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
2024-09-03 08:19:22.097151: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [3]:
puzzle = SlidePuzzle(4)
size = int(2e7)
batch_size = int(10000)

In [4]:
states = jax.vmap(puzzle.get_initial_state, in_axes=0)(key=jax.random.split(jax.random.PRNGKey(32),1))
#states = puzzle.State(board=jnp.array([0, 12, 9, 13, 15, 11, 10, 14, 3, 7, 2, 5, 4, 8, 6, 1], dtype=jnp.uint8))[jnp.newaxis, ...]
target = puzzle.State(board=jnp.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0], dtype=jnp.uint8))

In [5]:
print(states[0])
print(target)

┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃ 6 ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃ F ┃ 9 ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 3 ┃ 4 ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛
┏━━━┳━━━┳━━━┳━━━┓
┃ 1 ┃ 2 ┃ 3 ┃ 4 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 5 ┃ 6 ┃ 7 ┃ 8 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 9 ┃ A ┃ B ┃ C ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ D ┃ E ┃ F ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛


In [6]:
astar_fn = astar_builder(puzzle, SlidePuzzleHeuristic(puzzle).distance, batch_size, size)

states, filled = HashTable.make_batched(puzzle.State, states, batch_size)
astar_result, solved, solved_idx = astar_fn(states, filled, target)

In [7]:
print(solved, solved_idx)

True HashTableIdx_HeapValue(index=Array([7355726], dtype=int32), table_index=Array([0], dtype=int32))


In [8]:
solved_st = astar_result.hashtable.table[solved_idx.index, solved_idx.table_index][0]
solved_cost = astar_result.cost[solved_idx.index, solved_idx.table_index][0]
print(solved_st)
print(solved_cost)

┏━━━┳━━━┳━━━┳━━━┓
┃ 1 ┃ 2 ┃ 3 ┃ 4 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 5 ┃ 6 ┃ 7 ┃ 8 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 9 ┃ A ┃ B ┃ C ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ D ┃ E ┃ F ┃   ┃
┗━━━┻━━━┻━━━┻━━━┛
66.0


In [9]:
astar_result, solved, solved_idx = astar_fn(states, filled, target)

In [10]:
parants = astar_result.parant
table = astar_result.hashtable.table
cost = astar_result.cost

In [11]:
path = []
parant_last = parants[solved_idx.index, solved_idx.table_index][0]
for i in range(100):
    if parant_last[0] == -1:
        break
    path.append(parant_last)
    parant_last = parants[*parant_last]

for p in path[::-1]:
    state = table[p[0], p[1]]
    c = cost[p[0], p[1]]
    print(state)
    print(c)
print(solved_st)
print(solved_cost)

┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃ 6 ┃   ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃ F ┃ 9 ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 3 ┃ 4 ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛
0.0
┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃   ┃ 6 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃ F ┃ 9 ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 3 ┃ 4 ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛
1.0
┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃ 9 ┃ 6 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃ F ┃   ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 3 ┃ 4 ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛
2.0
┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃ 9 ┃ 6 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃   ┃ F ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 3 ┃ 4 ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛
3.0
┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃ 9 ┃ 6 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃ 4 ┃ F ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 3 ┃   ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛
4.0
┏━━━┳━━━┳━━━┳━━━┓
┃ C ┃ 1 ┃ B ┃ E ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ A ┃ 5 ┃ 9 ┃ 6 ┃
┣━━━╋━━━╋━━━╋━━━┫
┃ 8 ┃ 4 ┃ F ┃ D ┃
┣━━━╋━━━╋━━━╋━━━┫
┃   ┃ 3 ┃ 2 ┃ 7 ┃
┗━━━┻━━━┻━━━┻━━━┛
5.0
┏━━━